In [1]:
# init
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
import h5py
import numpy as np
from sklearn.model_selection import train_test_split
import yasa
#import matplotlib.pyplot as plt

X_train = h5py.File("X_train.h5", "r")
y_train = pd.read_csv("y_train.csv")

In [2]:
# Features
'''
def lissage(Ly,p):
    #Fonction qui débruite une courbe par une moyenne glissante
    #sur 2P+1 points
    Lyout=[]
    for index in range(p, len(Ly)-p):
        if index == p:
            average = np.mean(Ly[index-p : index+p+1])
        else:
            average -= Ly[index - 1] / (2 * p + 1)
            average += Ly[index + p] / (2 * p + 1)
        Lyout.append(average)
    return Lyout

def nbmax(a):
    N=len(a)
    M=len(a[0])
    res = np.zeros(N)
    for i in range(N):
        y = lissage(a[i], 20)
        nb=0
        e=y[0]
        f=y[1]
        g=y[2]
        for j in range(3,len(y)):
            e=f
            f=g
            g=y[j]
            if (f>e and f>g):
                nb+=1
        res[i] = nb
    return res
'''
def deltabeta(data):
    N = len(data)
    sf = 125
    res = np.zeros((N, 2))
    for i in range(N):
        band = yasa.bandpower(data[i, 11:], sf, kwargs_welch={})
        d = band["Delta"]["CHAN001"]
        b = band["Beta"]["CHAN001"]
        res[i][0] = d
        res[i][1] = d / b
    return res

def extract_features(h5):
    data = h5["features"][:]
    features = []
    features.append(data[:, :])
    #features.append(data[:, :11])
    features.append(data[:, 11:].max(1).reshape(-1, 1))
    features.append(data[:, 11:].min(1).reshape(-1, 1))
    features.append(np.abs(data[:, 11:]).mean(1).reshape(-1, 1))
    features.append(deltabeta(data))
    ##features.append(nbmax(data[:, 11:]).reshape(-1, 1))
    features = np.concatenate(features, 1)
    return features

In [3]:
#data = X_train["features"][:]
#deltabeta(data[:100])

In [4]:
# Train
features_train = extract_features(X_train)
# Données
#X_train,X_test,y_train,y_test=train_test_split(features_train,y_train["label"])

In [32]:
from sklearn.model_selection import cross_val_score
import lightgbm as lgb
#from sklearn.model_selection import train_test_split
#from sklearn.neural_network import MLPClassifier
#clf = RandomForestClassifier(n_estimators=100,n_jobs=2,max_features="sqrt",max_depth=4,min_samples_split=None,verbose=1)
#clf = RandomForestClassifier(n_estimators=10,verbose=1,n_jobs=-1)
clf = lgb.LGBMClassifier(n_estimators=500,num_leaves=10,learning_rate=0.1,random_state=42,verbose=1)
'''clf= MLPClassifier(hidden_layer_sizes=(100, ), activation='relu', solver='adam',\
                   alpha=0.0001, batch_size='auto', learning_rate='constant',\
                   learning_rate_init=0.001, power_t=0.5, max_iter=200, shuffle=True,\
                   random_state=None, tol=0.0001, verbose=False, warm_start=False,\
                   momentum=0.9, nesterovs_momentum=True, early_stopping=False,\
                   validation_fraction=0.1, beta_1=0.9, \
                   beta_2=0.999, epsilon=1e-08, n_iter_no_change=10)
'''
#from sklearn.neighbors import KNeighborsClassifier
#clf = KNeighborsClassifier(n_neighbors=5, weights='uniform', algorithm='auto', leaf_size=30, p=2, metric='minkowski',metric_params=None, n_jobs=None)
#from sklearn.svm import SVC
#clf = SVC(C=1.0, kernel='rbf', degree=3, gamma='auto', coef0=0.0, shrinking=True, probability=False, tol=0.001, cache_size=200, class_weight=None, verbose=False, max_iter=-1, decision_function_shape='ovr', random_state=None)
#scores=cross_val_score(clf, features_train, y_train, cv=3)
#clf = SGDClassifier(loss='hinge', penalty='l2', alpha=0.0001, l1_ratio=0.15, fit_intercept=True, max_iter=1000, tol=0.001, shuffle=True, verbose=0, epsilon=0.1, n_jobs=None, random_state=None, learning_rate='optimal', eta0=0.0, power_t=0.5, early_stopping=False, validation_fraction=0.1, n_iter_no_change=5, class_weight=None, warm_start=False, average=False)
#from sklearn.naive_bayes import GaussianNB
#clf = GaussianNB(priors=None, var_smoothing=1e-09)
clf.fit(features_train, y_train["label"].values)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=500, n_jobs=-1, num_leaves=10, objective=None,
               random_state=42, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0,
               verbose=1)

In [31]:
'''y_pred=clf.predict(X_test)
import numpy as np
print((np.array(y_test)))
matches=[int(y_pred[k]==np.array(y_test)[k]) for k in range(len(y_pred))]
print(np.sum(matches)/len(y_pred))'''

'y_pred=clf.predict(X_test)\nimport numpy as np\nprint((np.array(y_test)))\nmatches=[int(y_pred[k]==np.array(y_test)[k]) for k in range(len(y_pred))]\nprint(np.sum(matches)/len(y_pred))'

In [33]:
# test
#X_test = h5py.File("X_test.h5", "r")
#features_test = extract_features(X_test)
y_pred = clf.predict(features_test)
with open("y_benchmark_0944.csv", "w") as f:
    f.write("".join(["id,label\n"] + ["{},{}\n".format(i, y) for i, y in enumerate(y_pred)]))